select now() at time zone 'Europe/Moscow'

select now()::date или select CURRENT_DATE

Предположим, у нас есть дата и время какого-то события и мы хотим посмотреть, к какой дате оно относится для Москвы и для UTC.
```sql
with x as 
(
select '2018-12-31 21:00:00+00'::timestamp with time zone ts
)
select (ts at time zone 'Europe/Moscow')::date dt_msk,
        (ts at time zone 'UTC')::date dt_utc
from x
```
___

#### ФУНКЦИЯ EXTRACT()

возвращает тип double precision

***DAY***   Для значений timestamp это день месяца (1-31), для значений interval — число дней
```sql
SELECT EXTRACT(DAY FROM TIMESTAMP '2001-02-16 20:38:40')
```
***HOUR, MONTH, YEAR***

***ISOYEAR***

Год по недельному календарю ISO 8601, в который попадает дата (не применимо к интервалам). Год по недельному календарю ISO начинается с понедельника недели, в которой оказывается 4 января, так что в начале января или в конце декабря год по ISO может отличаться от года по григорианскому календарю. Подробнее об этом рассказывается в описании поля week.

***WEEK***

Номер недели в году по недельному календарю ISO 8601. По определению, недели ISO 8601 начинаются с понедельника, а первая неделя года включает 4 января этого года. Другими словами, первый четверг года всегда оказывается в первой неделе этого года.

В системе нумерации недель ISO первые числа января могут относиться к 52-й или 53-й неделе предыдущего года, а последние числа декабря — к первой неделе следующего года.

*Например, 2005-01-01 относится к 53-й неделе 2004 г., а 2006-01-01 — к 52-й неделе 2005 г., тогда как 2012-12-31 включается в первую неделю 2013 г.*

***CENTURY***

Первый век начался 0001-01-01 00:00:00, хотя люди в то время так и не считали. Это определение распространяется на все страны с григорианским календарём.

Века с номером 0 не было; считается, что 1 наступил после -1.

***DECADE***

десятилетие

***EPOCH***

Для значений timestamp with time zone это число секунд с 1970-01-01 00:00:00 UTC (может быть отрицательным); для значений date и timestamp это число секунд с 1970-01-01 00:00:00 по местному времени, а для interval — общая длительность интервала в секундах.

Преобразовать время эпохи обратно, в значение дата/время, с помощью to_timestamp можно так:
```sql
SELECT    to_timestamp(982384720.12)
```
***DOW***

День недели, считая с воскресенья (0) до субботы (6)

***DOY***

День года (1-365/366).

***ISODOW***

День недели, считая с понедельника (1) до воскресенья (7) Результат отличается от dow только для воскресенья. Такая нумерация соответствует ISO 8601.
___

#### ФУНКЦИЯ TO_CHAR()

Например, вы хотите вывести год, месяц и день со специфическим разделителем или получить текстовое наименование месяца или дня недели. По результату работы она очень близка к extract(), но больше нацелена именно на форматирование.

to_char(timestamp[date],text) = преобразует время в текст; результат to_char(current_timestamp, 'HH12:MI:SS')

to_char(interval, text) = преобразует интервал в текст; to_char(interval '15h 2m 12s', 'HH24:MI:SS')

Предположим, мы хотим вывести сегодняшнюю дату в формате "Hello! Today is #название дня недели год.название месяца.день#" текстом. Для этого нужно выполнить следующий код: 

select to_char(now(),'"Hello! Today is" DAY yyyy-Mon-dd')
___

#### ФУНКЦИЯ DATE_TRUNC()

Функция date_trunc() позволяет отсечь заданное время, дату или дату со временем до нужной точности.

Например, если мы хотим округлить текущее время-дату до минут, то можно вызвать
```sql
select date_trunc('minute',now())
```
___

#### МАТЕМАТИЧЕСКИЕ ОПЕРАТОРЫ

К любой дате можно прибавить (и вычесть из неё) целое число X и получить другую дату, которая больше (меньше) изначальной.
```sql
select '2019-01-01'::date + 10
```
Результат: '2019-01-11'

Это — дата на 10 дней позже 2019-01-01.

При добавлении (или вычитании) целого числа к дате Postgres учитывает переходы между месяцами и годами и даёт верный ответ, соответствующий календарю. Учитываются даже високосные годы.

Аналогично можно вычесть из одной даты другую и получить расстояние в днях между этими датами. При такой операции тоже будет честная разница по календарю.
```sql
select '2019-02-10'::date - '2017-03-01'::date
```
Результат: 711
___

## ***Строковые данные: основные типы***

**CHARACTER**

Cтрока фиксированной длины, дополненная пробелами.

Длина строки такого типа всегда одинакова и задаётся в скобках.

Например, в столбце character(5) всегда будет пять символов: строку большей длины туда вставить не получится, а строка меньшей длины будет дополняться ведущими пробелами. Слово "SQL" в таком столбце будет выглядеть как "  SQL".

Основной паттерн использования такого типа — универсальные справочники буквенных кодов, например код страны в стандарте ISO (RU, US, UK и т. д.).

**CHARACTER VARYING**

Строка ограниченной переменной длины.

Например, в столбце типа character varying(5) нельзя будет хранить строку большей длины, но могут быть любые строки с меньшей длиной.

Этот тип данных повсеместно используется для хранения данных, поскольку позволяет ограничить ввод, сохраняя при этом возможность иметь строки произвольной длины.

**TEXT**

Cтрока неограниченной длины.

Самый удобный тип для пользователя, но самый тяжеловесный для администратора баз данных, так как в строку можно записать любой текст.

Для удобства все текстовые поля в нашем датасете с доставками представлены типом text.
___

## ***ОПЕРАТОРЫ***

**СОЕДИНЕНИЕ СТРОК**

Oператор конкатенации строк — || (две вертикальные черты). Он позволяет объединять две и более строки. строка1 || строка2 || ... || строкаN

Результатом соединения любых типов строковых данных будет тип text.


Если вы соединяете любую строку и NULL, то результатом будет NULL. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует NULL, используйте оператор coalesce.

*Напишите SQL-запрос, который выведет следующее сообщение для каждого водителя по форме: Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#*
```sql
select 'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. Его контактный номер: '||COALESCE(d.phone, '-') msg
FROM sql.driver d
```

**UPPER() И LOWER()**

Функции upper(your_text) и lower(your_text) переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

Результат функций upper() и lower() — тоже строковый, а значит, к нему можно применять все функции, применимые к этому типу данных.

*Напишите запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре.*
```sql
select
      c.cust_id,
      lower(c.cust_name)
FROM sql.customer c
JOIN sql.shipment s on c.cust_id = s.cust_id
GROUP BY c.cust_id
HAVING COUNT(s.ship_id) > 10
ORDER BY 1
```

**REPLACE()**

С помощью функции replace() можно заменять символы в строках. Где, что, на что.

*Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: название_штата__название_города*
```sql
SELECT
      lower(replace(c.state, ' ', '_')||'__'||replace(c.city_name, ' ', '_')) utm
FROM sql.city c
ORDER BY 1
```

**LEFT() И RIGHT()**

Функции left(string,n) и right(string,n) оставляют n левых или правых символов от строки, поданной на вход.
```sql
select left('0123456789', - 2), right('0123456789', - 2)
```
Результат: 01234567 и 23456789 (в первом случае — восемь символов с «отрезанными» 89 и во втором случае — восемь символов с «отрезанными» 01)

*Представим, что к вам пришёл разработчик, который хочет сократить поле state в таблице city до четырёх символов, и попросил проверить, останeтся ли значения в нём уникальными. Чтобы ответить на этот вопрос, напишите SQL-запрос, который выведет первые четыре символа названия штата и количество уникальных названий штатов, которому они соответствуют. Оставьте только те, которые относятся к двум и более штатам. Добавьте сортировку по первому столбцу. Столбцы в выдаче: code (четыре первых символа в названии штата), qty (количество уникальных названий штата, начинающихся с этих символов).*
```sql
SELECT
      left(c.state, 4) code,
      COUNT(DISTINCT c.state) qty
FROM sql.city c
GROUP BY code
HAVING COUNT(DISTINCT c.state) >1
ORDER BY 1
```

**FORMAT()**

Функция format() используется для составления форматированного текста с подстановками. То же самое можно сделать через конкатенацию строк, но это неудобно и громоздко.

*Допустим, у нас есть шаблон "Hello, #Имя пользователя#!" и таблица водителей, которым нужно вывести приветствие.*
```sql
select format('Hello, %s!', d.first_name) from shipping.driver d
```
*Напишем запрос, который описывает содержимое каждой строки в таблице в виде текста.*
```sql
select format('driver_id = %s, first_name = %s, last_name = %s, address = %s, zip_code = %s, phone = %s, city_id = %s', driver_id, first_name, last_name, address, zip_code, phone, city_id) from shipping.driver d
```
Если в вашем шаблоне присутствует одинарная кавычка, то для удобства можно вместо одинарных кавычек использовать $$ (два знака доллара):
```sql
select $$ some_string with quotes ' $$
```
*Давайте подготовим географическую сводку для каждого города. Напишите SQL-запрос, который выведет описание региона в следующем формате: [city_name] is located in [state]. There's [population] people living there. Its area is [area]
```sql
SELECT
      format('%s is located in %s. There''s %s people living there. Its area is %s', c.city_name, c.state, c.population, c.area) str
FROM sql.city c
```